**Food Atlas Data**
The US Food Atlas provides county data across the United States on regions with lower food accessibility.
In parsing through these data, we must:
- Import the data to Python.
- Convert the data to a workable datatype (`df` (DataFrame), in this case).
- Extract Georgia-specific data.
- Identify and extract relevant variables from these data (spreadsheet provides description of variables).
- Visualize these data graphically (choropleth) enabling correlational comparisons to MTMG data.

Libraries are essentially pre-made toolkits equipped with functions and features we can use to (in this case) manipulate data without building these methods from scratch ourselves.
Importing relevant libraries:

In [ ]:
import json
import plotly
import pandas as pd
import plotly.io as pio
import geopandas as gpd
import matplotlib.pyplot as plt

from plotly import graph_objects as go

# Mapbox Access Token
pio.templates.default = "plotly"
mapbox_access_token = "pk.eyJ1Ijoibm5kbXRyZmYiLCJhIjoiY20xYmFtc2pvMGx1czJrb2ZqZHRqdHMxbCJ9.BghPzt_eZzDNAV12Lx54-w"

Loading the Excel file:
- Define the path to the file (the path to its location on my machine).
- Load the file in, specifically referencing the third worksheet, which is the sheet that contains relevant data (the index starts at 0).
- Extract all data in the column `State` with the keyword `Georgia`, eliminating case specificity from my query.
- Display the resulted dataframe.

In [ ]:
file_path = '/Users/anniem/Desktop/6802_DATA/Data/FoodAccessResearchAtlasData2019.xlsx'     # defining the path to the file.

df = pd.read_excel(file_path, sheet_name=2)                                                 # loading the third sheet.

filtered_df = df[df['State'].str.contains('georgia', case=False, na=False)]                 # parses through 'State' column for strings containing 'Georgia'.

# display(filtered_df)                                                                      # displays filtered dataframe.

Now that we've displayed our data, we have to consider what exactly we're looking at. The worksheet offers an extensive list of variables in the second sheet, so let's repeat the same process as above to take a look at them:

In [ ]:
vars_df = pd.read_excel(file_path, sheet_name=1)    # loading the second sheet.

pd.set_option('display.max_rows', None)             # change `15` to `None` to display all rows.
pd.set_option('display.max_colwidth', 80)           # change `80` to `None` to unwrap columns.

# display(vars_df)                                  # displays filtered dataframe.

Expectedly, this is *a lot* to parse through! Here are a few notable demographic variables I identified (not a comprehensive list):

- 146: `TractSNAP`
    - Total count of housing units receiving SNAP benefits in tract (Census Tract of region).
- 145: `TractHUNV`
    - Total count of housing units without a vehicle in region.
- 135: `TractLOWI`
    - Total count of low-income population in region.
- 133: `lasnap20`
    - Count of housing units receiving SNAP benefits count beyond 20 miles from supermarket.
- 131: `lahunv20`
    - Count of housing units without vehicle beyond 20 miles from supermarket.
- 115: `laseniors20`
    - Count of low-access seniors (65+) beyond 20 miles from supermarket.
- 113: `lakids20`
    - Count of low-access kids (0-17) beyond 20 miles from supermarket.

- These data are repeated for 10 miles and <10 miles (1-5 miles).

I chose these data specifically because they can give us an idea of the 'big picture' of location- and income-based food insecurity. More detailed demographic data bears significance, too, and I'll be looking at that later on once I get these larger regions mapped out.

As far as generating a preliminary choropleth goes, I'll be using:
- `LILATracts_1And10`
    - Flag for low-income and low-access tracts measured at 1 mile for urban areas and 10 miles for rural areas.
- `LILATracts_halfAnd10`
    - Flag for low-income and low-access tracts measured at 1/2 mile for urban areas and 10 miles for rural areas.
- `PovertyRate`
    - Share of the tract population living at or below the federal poverty thresholds for family size.
- `LAPOP1_10`
    - Population count beyond 1 mile for urban areas or 10 miles for rural areas from a supermarket.
- `TractLOWI`
    - Total count of low-income population in the tract.

... as potential datasources. Let's try that out:

Alternative Shapefile Datasets: 

https://www.sciencebase.gov/catalog/item/59fa9f5be4b0531197affb23

https://gagiohome-gagio.hub.arcgis.com/search?collection=dataset&sort=Date%20Updated%7Cmodified%7Cdesc

In [ ]:
# Load the GeoJSON file
geo_georgia = gpd.read_file('/Users/anniem/Desktop/6802_DATA/Data/georgia-with-county-boundaries_1092.geojson')
geo_georgia_json = json.loads(geo_georgia.to_json())

# Remove " County" suffix from the filtered DataFrame
filtered_df['County'] = filtered_df['County'].str.replace(' County', '', regex=False)

In [ ]:
# Create the figure object
fig1 = go.Figure(
    go.Choroplethmapbox(
        geojson=geo_georgia_json,
        featureidkey="properties.name",  # Match the GeoJSON key
        locations=filtered_df["County"],  # DataFrame column with counties
        z=filtered_df["LAPOP1_10"],  # Values to color
        zauto=True,
        colorscale='YlGnBu',
        showscale=True,
    )
)
# Blackbody,Bluered,Blues,C ividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portl and,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd.

# Update layout with Mapbox settings
fig1.update_layout(
    mapbox_accesstoken=mapbox_access_token,
    mapbox_style="carto-positron",
    mapbox_zoom=6, 
    mapbox_center={"lat": 32.5, "lon": -83.5},  # Center the map over Georgia
    title_x=0.5,
    title_y=0.95,
    title_text="Georgia Population Count Beyond 1 Mile (Urban) or 10 Miles (Rural) from a Supermarket",
    margin=dict(
                l=0,
                r=0,
                b=0,
                t=90,
                pad=4,
                autoexpand=True
    ),
    width=800,
    height=800
)

# Show the figure
fig1.show()


In [ ]:
# Create the figure object
fig1 = go.Figure(
    go.Choroplethmapbox(
        geojson=geo_georgia_json,
        featureidkey="properties.name",  # Match the GeoJSON key
        locations=filtered_df["County"],  # DataFrame column with counties
        z=filtered_df["LILATracts_halfAnd10"],  # Values to color
        zauto=True,
        colorscale='tealgrn',
        showscale=True,
    )
)
# Blackbody,Bluered,Blues,C ividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portl and,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd.

# Update layout with Mapbox settings
# low-income and low-access tracts measured at 1/2 mile for urban areas and 10 miles for rural areas
fig1.update_layout(
    mapbox_accesstoken=mapbox_access_token,
    mapbox_style="carto-positron",
    mapbox_zoom=6,
    mapbox_center={"lat": 32.5, "lon": -83.5},  # Center the map over Georgia
    title_x=0.5,
    title_y=0.95,
    title_text="Low-Income and Low-Access Tracts Beyond 1/2 Mile (Urban) or 10 Mile (Rural) from a Supermarket",
    margin=dict(
                l=0,
                r=0,
                b=0,
                t=90,
                pad=4,
                autoexpand=True
    ),
    width=900,
    height=800
)

# Show the figure
fig1.show()


In [ ]:
# Create the figure object
fig1 = go.Figure(
    go.Choroplethmapbox(
        geojson=geo_georgia_json,
        featureidkey="properties.name",  # Match the GeoJSON key
        locations=filtered_df["County"],  # DataFrame column with counties
        z=filtered_df["TractLOWI"],         # Values to color
        zauto=True,
        colorscale='YlGnBu',
        showscale=True,
    )
)
# Blackbody,Bluered,Blues,C ividis,Earth,Electric,Greens,Greys,Hot,Jet,Picnic,Portl and,Rainbow,RdBu,Reds,Viridis,YlGnBu,YlOrRd.

# Update layout with Mapbox settings
# low-income and low-access tracts measured at 1/2 mile for urban areas and 10 miles for rural areas
fig1.update_layout(
    mapbox_accesstoken=mapbox_access_token,
    mapbox_style="carto-positron",  
    mapbox_zoom=6,
    mapbox_center={"lat": 32.5, "lon": -83.5},  # Center the map over Georgia
    title_x=0.5,
    title_y=0.95,
    title_text="Total Count of Low Income Population in Tract",
    margin=dict(
                l=0,
                r=0,
                b=0,
                t=90,
                pad=4,
                autoexpand=True
    ),
    width=900,
    height=800
)

# Show the figure
fig1.show()
